In [1]:
import numpy as np
import pandas as pd
import re

import os
import yaml
import torch
from torch import package
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc)
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from yargy import Parser
from yargy.relations import gnc_relation
from yargy.predicates import gram
from yargy.pipelines import morph_pipeline
from yargy.interpretation import (
    fact,
    attribute
)

from yargy import (
    Parser,
    rule, or_,
    not_, and_
)
from yargy.interpretation import fact
from IPython.display import display
import pymorphy2
from yargy import Parser, rule, and_
from yargy.predicates import gram, is_capitalized, dictionary
from yargy.tokenizer import (
    QUOTES,
    LEFT_QUOTES,
    RIGHT_QUOTES,

    MorphTokenizer,
    TokenRule
)
import warnings
import re
warnings.filterwarnings('ignore')

In [2]:
#!wget https://storage.yandexcloud.net/natasha-navec/packs/news_v1_1B_250K_300d_100q.tar
#!wget https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar

In [3]:
data = pd.read_csv('test_data.csv')

In [4]:
data

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...
477,5,140,client,Спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте


In [5]:
# Проверяем что менеджер поздаровался и попрощался с клиентом
def is_manager_good(greetings,goodbye):
    return 1 if (len(greetings) != 0 and len(goodbye) != 0) else 0

In [6]:
class RuleBased:
    """
    Based on rules. 
    """
    def __init__(self,corpus, rules=None):
        if not rules:
            self.rules = {'greetings': ['здравствуйте', "добрый день",
                                        "привет","добрый", "приветсвую","здрасьте"],
                         'introduce': ['меня зовут',"мое имя","обращайтесь ко мне",
                                       "можете звать меня", "зовите меня","называйте меня",
                                       'меня зовут','меня',"да это"],
                         'company_name':["компании","компания",'компанию'],
                         'goodbye':['досвидания',"до свидания","пока",
                                    "всего хоршего","всего доброго","прощайте","всех благ",
                                   "хорошего"]}
        else:
            self.rules = rules
        self.greetings = {}
        self.goodbye = {}
        self.manager_name = {}
        self.company_name = {}        
        self.manager_introduce = {}
        self.data = corpus
        self.geeting_goodbye = {}
        self.data['greeting_goodbye'] = None
        self._diag()
        
    def _diag(self):
        self.d_count = len(self.data.groupby(by='dlg_id'))
        self.dialogs = self.data.groupby(by=['dlg_id','role'])
    
    
    # Проверям поздоровался ли манагер
    def fing_greetings(self,dig):
        manager_greetings = dig[dig.text.str.lower().str.contains('|'.\
                                                                  join(self.rules['greetings']))]
        return manager_greetings
    
    
    # Проверяем попращался ли манагер
    def find_goodbye(self,dig):
        manager_goodbye = dig[dig.text.str.lower().str.contains('|'.\
                                                                join(self.rules['goodbye']))]
        return manager_goodbye
    
    
    # Ищем где представился манагер
    def find_introduce(self,dig):
        manager_name = dig[dig.text.str.lower().str.contains('|'.\
                                                             join(self.rules['introduce']))]
        
        return manager_name
    
    #Ищем имя манагера
    def find_name(self,dig):
        manager_name = dig[dig.text.str.lower().str.contains('|'.\
                                                             join(self.rules['introduce']))][:1]
        start = []
        l = []
        for x in self.rules['introduce']:
            if manager_name.text.str.lower().str.find(f'{x}').values > -1:
                start.append(manager_name.text.str.lower().str.find(f'{x}').values)
                l.append(len(x))
        
        string  = manager_name.text.str[start[0][0] + l[0] + 1:].values[0].split()
        
        return ''.join(string[0]).capitalize()
    
    
    # Ищем название компании
    def find_company(self,dig):
        company_name = dig[dig.text.str.lower().str.contains('|'.\
                                                join(self.rules['company_name']))][:1]
        
        start = [company_name.text.str.lower().str.find(f'{x}').values for x \
                 in self.rules['company_name'] if company_name.text.str.find(f'{x}').values > -1]

        string  = company_name.text.str[start[0][0]+9:].values[0].split()
    
        morph = pymorphy2.MorphAnalyzer()
        name = []
        for i in range(len(string)):
            if morph.parse(string[i])[0].normal_form == string[i] and 'ADVB' not in morph.parse(string[i])[0].tag:
                name.append(string[i])
            else:
                break
        return ' '.join(name).capitalize()
    
    #Получаем датафрейм здраровонье и прощание
    def get_greetings_goodbye(self):
        """
        make two dataframes: greetings and goodbyes. fill column greeting_goodbye
        return: DataFrame of greetings and DataFrame of goodbyes 
        """
        for i in range(self.d_count):
            self.greetings[i] = self.fing_greetings(self.dialogs.get_group((i,'manager')))
            self.goodbye[i] = self.find_goodbye(self.dialogs.get_group((i,'manager')))
            self.geeting_goodbye[i] = is_manager_good(self.greetings.get(i,[]),
                                                      self.goodbye.get(i,[]))
            self.data.loc[self.data.dlg_id == i ,'greeting_goodbye'] =  is_manager_good(
                self.greetings.get(i,[]),self.goodbye.get(i,[]))

                

        return pd.concat(self.greetings,ignore_index=True)[['dlg_id','text']],pd.concat(self.goodbye,\
                                                                     ignore_index=True)[['dlg_id','text']]
        
     #Получаем датафрейм представления  
    def get_manager_inroduce(self):
        for i in range(self.d_count):
            self.manager_introduce[i] = self.find_introduce(self.dialogs.get_group((i,'manager')))
        return pd.concat(self.manager_introduce,ignore_index=True)[['dlg_id','text']]
    
    #Получаем поздаровался и попрощался ли манагер 
    def get_manager_stats(self):
        if len(self.greetings) == 0:
            self.get_greetings_goodbye()
        return pd.DataFrame(self.geeting_goodbye.values(),
                            index = self.geeting_goodbye.keys(),columns=['greeting_goodbye'])
    
    #Получаем имя манагера 
    def get_manager_name(self):
        for i in range(self.d_count):
            try:
                self.manager_name[i] = self.find_name(self.dialogs.get_group((i,'manager')))  
            except:
                self.manager_name[i] = None
        return pd.DataFrame(self.manager_name.values(),
                            index = self.manager_name.keys(),columns=['manager_name'])
    #Получаем имя компании 
    def get_company_name(self):
        for i in range(self.d_count):
            try:
                self.company_name[i] = self.find_company(self.dialogs.get_group((i,'manager')))
            except:
                self.company_name[i] = None
        return pd.DataFrame(self.company_name.values(),
                            index = self.company_name.keys(),columns=['company_name'])

In [7]:
%time r = RuleBased(data)

%time g,b = r.get_greetings_goodbye()
%time i = r.get_manager_inroduce()
%time n = r.get_manager_name()
%time c = r.get_company_name()
%time s = r.get_manager_stats()

CPU times: user 2.07 ms, sys: 0 ns, total: 2.07 ms
Wall time: 1.69 ms
CPU times: user 21.8 ms, sys: 6.27 ms, total: 28.1 ms
Wall time: 25.2 ms
CPU times: user 7.75 ms, sys: 0 ns, total: 7.75 ms
Wall time: 7.17 ms
CPU times: user 26.9 ms, sys: 1.39 ms, total: 28.3 ms
Wall time: 27.5 ms
CPU times: user 389 ms, sys: 84.2 ms, total: 473 ms
Wall time: 472 ms
CPU times: user 217 µs, sys: 92 µs, total: 309 µs
Wall time: 335 µs


In [8]:
g

,dlg_id,text
0,0,Алло здравствуйте
1,1,Алло здравствуйте
2,2,Алло здравствуйте
3,3,Алло дмитрий добрый день
4,3,Добрый меня максим зовут компания китобизнес у...


In [9]:
b

,dlg_id,text
0,0,Всего хорошего до свидания
1,1,Угу да вижу я эту почту хорошо тогда исправлю ...
2,1,До свидания
3,3,Угу все хорошо да понедельника тогда всего доб...
4,4,Во вторник все ну с вами да тогда до вторника ...
5,5,Ну до свидания хорошего вечера


In [10]:
i

,dlg_id,text
0,0,Меня зовут ангелина компания диджитал бизнес з...
1,1,Меня зовут ангелина компания диджитал бизнес з...
2,2,Меня зовут ангелина компания диджитал бизнес з...
3,3,Добрый меня максим зовут компания китобизнес у...
4,5,Да это анастасия


In [11]:
n

,manager_name
0,Ангелина
1,Ангелина
2,Ангелина
3,Максим
4,None
5,Анастасия


In [12]:
c

,company_name
0,Диджитал бизнес
1,Диджитал бизнес
2,Диджитал бизнес
3,Китобизнес
4,None
5,None


In [13]:
s

,greeting_goodbye
0,1
1,1
2,0
3,1
4,0
5,0


In [14]:
def make_first_lowercase(x):
    return ''.join(x.split()[0].lower()) + ' ' + ' '.join(x.split()[1:])
class Natasha:
        """
    Based on natasha. 
    """
    def __init__(self,corpus,path_novec='navec_news_v1_1B_250K_300d_100q.tar',path_ner='slovnet_ner_news_v1.tar',rules = None):
        if not rules:
            self.rules = {'greetings': ['здравствуйте', "добрый день",
                                        "привет","добрый", "приветсвую","здрасьте"],
                         'introduce': ['меня зовут',"мое имя","обращайтесь ко мне",
                                       "можете звать меня", "зовите меня","называйте меня",
                                       'меня зовут','меня',"да это"],
                         'company_name':["компании","компания",'компанию'],
                         'goodbye':['досвидания',"до свидания","пока",
                                    "всего хоршего","всего доброго","прощайте","всех благ",
                                   "хорошего"]}
        else:
            self.rules = rules
            

        
        self.data = corpus
        self.data['greeting_goodbye'] = None
        self.navec = Navec.load(path_novec)
        self.ner = NER.load(path_ner)
        self.ner.navec(self.navec)
        self.data = corpus.copy()
        self.greetings = {}
        self.goodbye = {}
        self.manager_name = {}
        self.company_name = {}        
        self.manager_introduce = {}
        self.geeting_goodbye = {}
        self.names = {}
        self.greeting = {}
        self.company = {}
        self._init_silero()
        self.data.text = self.data.text.str.replace('+','plus').str.lower().apply(self.make_capitalize).apply(make_first_lowercase)
        self._diag()
        
        
    def _diag(self):
        self.d_count = len(self.data.groupby(by='dlg_id'))
        self.dialogs = self.data.groupby(by=['dlg_id','role'])
        
        
    def _init_silero(self,pre_download=True):
        if not pre_download:
            torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                                   'latest_silero_models.yml',
                                   progress=False)
        with open('latest_silero_models.yml', 'r') as yaml_file:
            models = yaml.load(yaml_file, Loader=yaml.SafeLoader)
        model_conf = models.get('te_models').get('latest')
        model_url = model_conf.get('package')

        model_dir = "downloaded_model"
        os.makedirs(model_dir, exist_ok=True)
        model_path = os.path.join(model_dir, os.path.basename(model_url))
        
        pack = package.PackageImporter(model_path)
        self.model = pack.load_pickle("te_model", "model")
        
        
    #Делаем энтити с большой буквы(ОЧЕНЬ долго работает) 
    def make_capitalize(self,text, lan='ru'):
        return self.model.enhance_text(text, lan)
        
    #Получаем имя манагера 
    def get_manager_name(self):
        for i in range(self.d_count):
            data = self.dialogs.get_group((i,'manager'))
            tmp = data.text
            string = ' '.join(tmp)
            markup = self.ner(string)
            name_spans = [string[x.start:x.stop] for x in markup.spans if x.type=='PER' and re.search(f"{'|'.join(self.rules['introduce'])}",string[x.start- 15 if x.start-15 > 0 else 0:x.stop])]
            if len(name_spans) > 0:
                self.names[i] = name_spans[0] 
            else:
                self.names[i] = None
        return pd.DataFrame(self.names.values(),
                        index = self.names.keys(),columns=['manager_name'])
    # Проверям поздоровался ли манагер
    def fing_greetings(self,dig):
        manager_greetings = dig[dig.text.str.lower().str.contains('|'.\
                                                                  join(self.rules['greetings']))]
        return manager_greetings
    
    
    # Проверяем попращался ли манагер
    def find_goodbye(self,dig):
        manager_goodbye = dig[dig.text.str.lower().str.contains('|'.\
                                                                join(self.rules['goodbye']))]
        return manager_goodbye
    
    
    # Ищем поздаровался ли манагер
    def find_introduce(self,dig):
        manager_name = dig[dig.text.str.lower().str.contains('|'.\
                                                             join(self.rules['introduce']))]
        
        return manager_name
    
    
    # Ищем название компании
    def find_company(self,dig):
        company_name = dig[dig.text.str.lower().str.contains('|'.\
                                                join(self.rules['company_name']))][:1]
        
        start = [company_name.text.str.lower().str.find(f'{x}').values for x \
                 in self.rules['company_name'] if company_name.text.str.find(f'{x}').values > -1]

        string  = company_name.text.str[start[0][0]+9:].values[0].split()
    
        morph = pymorphy2.MorphAnalyzer()
        name = []
        for i in range(len(string)):
            if morph.parse(string[i])[0].normal_form == string[i] and 'ADVB' not in morph.parse(string[i])[0].tag:
                name.append(string[i])
            else:
                break
        return ' '.join(name).capitalize()
    
    
    def get_greetings_goodbye(self):
        """
        make two dataframes: greetings and goodbyes. fill column greeting_goodbye
        return: DataFrame of greetings and DataFrame of goodbyes 
        """
        for i in range(self.d_count):
            self.greetings[i] = self.fing_greetings(self.dialogs.get_group((i,'manager')))
            self.goodbye[i] = self.find_goodbye(self.dialogs.get_group((i,'manager')))
            self.geeting_goodbye[i] = is_manager_good(self.greetings.get(i,[]),
                                                      self.goodbye.get(i,[]))
            self.data.loc[self.data.dlg_id == i ,'greeting_goodbye'] =  is_manager_good(
                self.greetings.get(i,[]),self.goodbye.get(i,[]))

                

        return pd.concat(self.greetings,ignore_index=True)[['dlg_id','text']],pd.concat(self.goodbye,\
                                                                     ignore_index=True)[['dlg_id','text']]
        
        
    def get_manager_inroduce(self):
        for i in range(self.d_count):
            self.manager_introduce[i] = self.find_introduce(self.dialogs.get_group((i,'manager')))
        return pd.concat(self.manager_introduce,ignore_index=True)[['dlg_id','text']]
    
    
    def get_manager_stats(self):
        if len(self.greetings) == 0:
            self.get_greetings_goodbye()
        return pd.DataFrame(self.geeting_goodbye.values(),
                            index = self.geeting_goodbye.keys(),columns=['greeting_goodbye'])
    
    
    def get_company_name(self):
        for i in range(self.d_count):
            try:
                self.company_name[i] = self.find_company(self.dialogs.get_group((i,'manager')))
            except:
                self.company_name[i] = None
        return pd.DataFrame(self.company_name.values(),
                            index = self.company_name.keys(),columns=['company_name'])
    
        

In [15]:
%time nat = Natasha(data)

%time g,b = nat.get_greetings_goodbye()
%time i = nat.get_manager_inroduce()
%time n = nat.get_manager_name()
%time c = nat.get_company_name()
%time s = nat.get_manager_stats()

CPU times: user 1min 22s, sys: 1.33 s, total: 1min 23s
Wall time: 24.6 s
CPU times: user 19.6 ms, sys: 0 ns, total: 19.6 ms
Wall time: 19.4 ms
CPU times: user 8.26 ms, sys: 6 µs, total: 8.27 ms
Wall time: 7.77 ms
CPU times: user 605 ms, sys: 1.2 s, total: 1.81 s
Wall time: 263 ms
CPU times: user 528 ms, sys: 508 ms, total: 1.04 s
Wall time: 420 ms
CPU times: user 308 µs, sys: 14 µs, total: 322 µs
Wall time: 332 µs


In [16]:
g

,dlg_id,text
0,0,алло здравствуйте!
1,1,алло здравствуйте!
2,2,алло здравствуйте!
3,3,алло Дмитрий добрый день.
4,3,добрый меня Максим зовут компания Китобизнес у...


In [17]:
b

,dlg_id,text
0,0,всего хорошего до свидания.
1,1,"угу, Да вижу я эту почту хорошо тогда исправлю..."
2,1,до свидания.
3,3,угу все хорошо да понедельника тогда всего доб...
4,4,во вторник все ну с вами да тогда до вторника ...
5,5,ну до свидания хорошего вечера?


In [18]:
i

,dlg_id,text
0,0,"меня зовут Ангелина, компания диджитал бизнес,..."
1,1,"меня зовут Ангелина, компания диджитал бизнес ..."
2,2,"меня зовут Ангелина, компания диджитал бизнес ..."
3,3,добрый меня Максим зовут компания Китобизнес у...
4,5,да это Анастасия!


In [19]:
n

,manager_name
0,Ангелина
1,Ангелина
2,Ангелина
3,Максим
4,None
5,Анастасия


In [30]:
c

,manager_name
0,диджитал бизнес
1,диджитал бизнес
2,диджитал бизнес
3,китобизнес
4,None
5,None


In [21]:
s

,greeting_goodbye
0,1
1,1
2,0
3,1
4,0
5,0


In [22]:
class Yargyparser:
    """
    Based on Yargy rules. 
    """
    def __init__(self,data,rules=None):
        if not rules:
            self.rules = {'greetings': ['здравствуйте', "добрый день",
                                        "привет","добрый", "приветсвую","здрасьте"],
                         'introduce': ['меня зовут',"мое имя","обращайтесь ко мне",
                                       "можете звать меня", "зовите меня","называйте меня",
                                       'меня зовут','меня',"да это"],
                         'company_name':["компании","компания",'компанию'],
                         'goodbye':['досвидания',"до свидания","пока",
                                    "всего хоршего","всего доброго","прощайте","всех благ",
                                   "хорошего"]}
        else:
            self.rules = rules
        self.data = data.copy()
        Name = fact(
            'Name',
            ['first', 'middle'])
        gnc = gnc_relation()
        
        FIRST = gram('Name').interpretation(
            Name.first.inflected()
        ).match(gnc)
        
        MIDDLE = gram('Patr').interpretation(
            Name.middle.inflected()
        ).match(gnc)
        
        NAME = or_(
            rule(
                FIRST,
                MIDDLE
            ),
            rule(
                FIRST)
        ).interpretation(
            Name
        )
        Cname = fact(
            'Cname',
            ['title', 'second']
        )
        TITLE = gram('NOUN').interpretation(
            Cname.title.inflected()
        ).match(gnc)
        SECOND = gram('NOUN').interpretation(
            Cname.second.inflected()
        ).match(gnc)
        CNAME = or_(
            rule(
                TITLE,
                SECOND,
            ),
            rule(
                TITLE
            )
        ).interpretation(
            Cname
        )
        Manager = fact(
            'Manager',
            ['type', 'name']
        )

        TYPE = morph_pipeline([
            'меня зовут',
            "мое имя",
            "а меня",
            "меня",
            "зовите меня",
            "я",
            "это"
        ]).interpretation(
            Manager.type.normalized()
        )
        
        MANAGER = or_(
            NAME,
            TITLE
        ).interpretation(
            Manager.name
        )

        MANAGER = rule(
            TYPE,
            MANAGER
        ).interpretation(
            Manager
        )  
        Company = fact(
            'Company',
            ['type', 'name']
        )


        CTYPE = morph_pipeline([
            'компания',
            'компании',
            "компанию",
        ]).interpretation(
            Company.type
        )

        COMPANY = or_(
            CNAME,
            TITLE
        ).interpretation(
            Company.name
        )

        COMPANY = rule(
            CTYPE,
            COMPANY
        ).interpretation(
            Company
        )
        Proxy = fact('Proxy', ['value'])

        RULES = or_(
            MANAGER,
            COMPANY
        ).interpretation(Proxy.value).interpretation(Proxy)
        self.TOKENIZER = MorphTokenizer()
        self.parser = Parser(RULES, tokenizer=self.TOKENIZER)
        self.greetings = {}
        self.goodbye = {}
        self.manager_name = {}
        self.company_name = {}        
        self.manager_introduce = {}
        self.geeting_goodbye = {}
        self.names = {}
        self.greeting = {}
        self.company = {}
        self.data['greeting_goodbye'] = None
        self._diag()
        
        

    def _diag(self):
        self.d_count = len(self.data.groupby(by='dlg_id'))
        self.dialogs = self.data.groupby(by=['dlg_id','role'])

    def find_names(self,dig):
        text = ' '.join(dig.text.str.lower())
        name,company = None,None
        for match in self.parser.findall(text):
            if type(match.fact.value.name).__name__ == 'Name':
                if match.fact.value.name.middle == None:
                    name = ''.join(match.fact.value.name.first)
                else:
                    name = ' '.join(match.fact.value.name)
            elif type(match.fact.value.name).__name__ == 'Cname':
                if match.fact.value.name.second == None:
                    company = ''.join(match.fact.value.name.title)
                else:
                    company = ' '.join(match.fact.value.name)
        return name,company
    def get_manager_name(self):
        for i in range(self.d_count):
            self.manager_name[i] = self.find_names(self.dialogs.get_group((i,'manager')))[0]
        return pd.DataFrame(self.manager_name.values(),
                        index = self.manager_name.keys(),columns=['manager_name'])
    def get_company_name(self):
        for i in range(self.d_count):
            self.manager_name[i] = self.find_names(self.dialogs.get_group((i,'manager')))[1]
        return pd.DataFrame(self.manager_name.values(),
                        index = self.manager_name.keys(),columns=['manager_name'])
    def fing_greetings(self,dig):
        manager_greetings = dig[dig.text.str.lower().str.contains('|'.\
                                                                  join(self.rules['greetings']))]
        return manager_greetings
    
    
    # Проверяем попращался ли манагер
    def find_goodbye(self,dig):
        manager_goodbye = dig[dig.text.str.lower().str.contains('|'.\
                                                                join(self.rules['goodbye']))]
        return manager_goodbye
    
    
    # Ищем имя манагера
    def find_introduce(self,dig):
        manager_name = dig[dig.text.str.lower().str.contains('|'.\
                                                             join(self.rules['introduce']))]
        
        return manager_name
    
    
    # Ищем название компании
    def find_company(self,dig):
        company_name = dig[dig.text.str.lower().str.contains('|'.\
                                                join(self.rules['company_name']))][:1]
        
        start = [company_name.text.str.lower().str.find(f'{x}').values for x \
                 in self.rules['company_name'] if company_name.text.str.find(f'{x}').values > -1]

        string  = company_name.text.str[start[0][0]+9:].values[0].split()
    
        morph = pymorphy2.MorphAnalyzer()
        name = []
        for i in range(len(string)):
            if morph.parse(string[i])[0].normal_form == string[i] and 'ADVB' not in morph.parse(string[i])[0].tag:
                name.append(string[i])
            else:
                break
        return ' '.join(name).capitalize()
    
    
    def get_greetings_goodbye(self):
        """
        make two dataframes: greetings and goodbyes. fill column greeting_goodbye
        return: DataFrame of greetings and DataFrame of goodbyes 
        """
        for i in range(self.d_count):
            self.greetings[i] = self.fing_greetings(self.dialogs.get_group((i,'manager')))
            self.goodbye[i] = self.find_goodbye(self.dialogs.get_group((i,'manager')))
            self.geeting_goodbye[i] = is_manager_good(self.greetings.get(i,[]),
                                                      self.goodbye.get(i,[]))
            self.data.loc[self.data.dlg_id == i ,'greeting_goodbye'] =  is_manager_good(
                self.greetings.get(i,[]),self.goodbye.get(i,[]))

                

        return pd.concat(self.greetings,ignore_index=True)[['dlg_id','text']],pd.concat(self.goodbye,\
                                                                     ignore_index=True)[['dlg_id','text']]
        
        
    def get_manager_inroduce(self):
        for i in range(self.d_count):
            self.manager_introduce[i] = self.find_introduce(self.dialogs.get_group((i,'manager')))
        return pd.concat(self.manager_introduce,ignore_index=True)[['dlg_id','text']]
    
    
    def get_manager_stats(self):
        if len(self.greetings) == 0:
            self.get_greetings_goodbye()
        return pd.DataFrame(self.geeting_goodbye.values(),
                            index = self.geeting_goodbye.keys(),columns=['greeting_goodbye'])

In [23]:

%time ya = Yargyparser(data) 

%time g,b = ya.get_greetings_goodbye()
%time i = ya.get_manager_inroduce()
%time n = ya.get_manager_name()
%time c = ya.get_company_name()
%time s = ya.get_manager_stats()


CPU times: user 74.8 ms, sys: 12 ms, total: 86.8 ms
Wall time: 85.7 ms
CPU times: user 14.2 ms, sys: 103 µs, total: 14.3 ms
Wall time: 13.3 ms
CPU times: user 7.42 ms, sys: 22 µs, total: 7.44 ms
Wall time: 7.01 ms
CPU times: user 334 ms, sys: 11.7 ms, total: 346 ms
Wall time: 344 ms
CPU times: user 231 ms, sys: 8.12 ms, total: 239 ms
Wall time: 239 ms
CPU times: user 548 µs, sys: 0 ns, total: 548 µs
Wall time: 565 µs


In [24]:
g

,dlg_id,text
0,0,Алло здравствуйте
1,1,Алло здравствуйте
2,2,Алло здравствуйте
3,3,Алло дмитрий добрый день
4,3,Добрый меня максим зовут компания китобизнес у...


In [25]:
b

,dlg_id,text
0,0,Всего хорошего до свидания
1,1,Угу да вижу я эту почту хорошо тогда исправлю ...
2,1,До свидания
3,3,Угу все хорошо да понедельника тогда всего доб...
4,4,Во вторник все ну с вами да тогда до вторника ...
5,5,Ну до свидания хорошего вечера


In [26]:
i

,dlg_id,text
0,0,Меня зовут ангелина компания диджитал бизнес з...
1,1,Меня зовут ангелина компания диджитал бизнес з...
2,2,Меня зовут ангелина компания диджитал бизнес з...
3,3,Добрый меня максим зовут компания китобизнес у...
4,5,Да это анастасия


In [27]:
n

,manager_name
0,ангелина
1,ангелина
2,ангелина
3,максим
4,None
5,анастасия


In [28]:
c

,manager_name
0,диджитал бизнес
1,диджитал бизнес
2,диджитал бизнес
3,китобизнес
4,None
5,None


In [29]:
s

,greeting_goodbye
0,1
1,1
2,0
3,1
4,0
5,0
